In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoModel, AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
import torch
import torch.nn as nn
import pickle
# from helper import get_labeled_answers, get_predicted_answers

In [31]:
tokenizer = AutoTokenizer.from_pretrained('KB/bert-base-swedish-cased')

In [32]:
model = torch.load('../results/model_CAR_3e_weighted_loss.pkl')
with open(r'../data/CAR/tokenized_CAR_data_eval.pkl', "rb") as input_file:
    test_data = pickle.load(input_file)
tokens = tokenizer.convert_ids_to_tokens(test_data[0]["input_ids"])
print(tokens)
dec = tokenizer.decode(test_data[0]["input_ids"])
print(dec)

['[CLS]', 'Eget', 'företag', 'Efter', 'beslut', 'Beslutet', 'skickas', 'till', 'den', 'ambassad', 'eller', 'generalkonsul', '##at', 'som', 'du', 'valde', 'i', 'webb', '##ansökan', '.', 'När', 'du', 'ska', 'hämta', 'ditt', 'beslut', 'ska', 'du', 'ta', 'med', 'ditt', 'pass', '.', 'Du', 'kan', 'få', 'uppehåll', '##stillstånd', 'för', 'två', 'år', 'men', 'aldrig', 'längre', 'än', 'ditt', 'pass', 'är', 'giltigt', '.', 'Om', 'du', 'får', 'uppehåll', '##stillstånd', 'för', 'mer', 'än', 'tre', 'månader', 'får', 'du', 'ett', 'uppehåll', '##stillstånd', '##skort', '.', 'Kort', '##et', 'är', 'ett', 'bevis', 'på', 'att', 'du', 'har', 'tillstånd', 'att', 'vara', 'i', 'Sverige', 'och', 'innehåller', 'bland', 'annat', 'dina', 'fingeravtryck', 'och', 'foto', 'på', 'dig', '.', 'Uppehåll', '##stillstånd', '##skort', '##et', 'tillverkas', 'i', 'samband', 'med', 'att', 'beslutet', 'fattas', ',', 'dock', 'tidigast', 'tre', 'månader', 'innan', 'uppehåll', '##stillståndet', 'börjar', 'gälla', '.', 'Det', 'gå

In [33]:
# TODO: check why this function cannot be exported!!
def compare_labels_and_output(output, labels, tokens):
    labels_stats = []
    for idx, label in enumerate(labels):
        if label == 1:
            count = 1
            next_label = labels[idx+1]
            while idx+count < len(labels) and next_label in [2, -100]:
                count += 1
                next_label = labels[idx+count]
            labels_stats.append((idx, count))
    print(labels_stats)
    num_matches = 0
    for ans in labels_stats:
        ans_start = ans[0]
        is_match = True
        answer_text = []
        for i in range(ans[1]):
            answer_text.append(tokens[ans_start+i])
            if output[ans_start+i] != labels[ans_start+i]:
                is_match = False
        if is_match:
            num_matches += 1
            print('found match: ', ' '.join(answer_text))
    return len(labels_stats), num_matches


def get_labeled_answers(labels, tokens):
    labels_stats = []
    for idx, label in enumerate(labels):
        if label == 1 and idx+1 < len(labels):
            next_label = labels[idx+1]
            count = 2
            while idx+count < len(labels) and next_label in [2, -100]:
                next_label = labels[idx+count]
                count += 1
            labels_stats.append((idx, count-1))
    print(labels_stats)
    all_answers = []
    for ans in labels_stats:
        ans_start = ans[0]
        answer_text = []
        for i in range(ans[1]):
            answer_text.append(tokens[ans_start+i])
        answer = ' '.join(answer_text)
        all_answers.append(answer)
    print('Answers: ', all_answers)


In [34]:
# Output class
# https://huggingface.co/docs/transformers/main_classes/output#transformers.modeling_tf_outputs.TFTokenClassifierOutput

model.eval()
test_input = []
test_labels = []
test_attn = []
token_type_ids = []
for i in range(len(test_data)):
    test_input.append(test_data[i]['input_ids'])
    test_labels.append(test_data[i]['labels'])
    test_attn.append(test_data[i]['attention_mask'])
    token_type_ids.append(test_data[i]['token_type_ids'])

print('Num test instances: ', len(test_input))
num_correct = 0
num_predicted = 0
num_pos_data = 0
for i in range(len(test_data)):
    output = model(torch.tensor([test_data[i]['input_ids']]), attention_mask=torch.tensor([test_data[i]['attention_mask']]), token_type_ids=torch.tensor([test_data[i]['token_type_ids']]), labels=torch.tensor([test_data[i]['labels']]))
    print('test idx: ', i)
    print('instance loss: ', output.loss)
    # print(output.logits)
    m = nn.Softmax(dim=2)
    max = m(output.logits)
    out = torch.argmax(max, dim=2)
    # print(max)
    # print('Output: ', out)
    # print('Labels: ', test_data[i]['labels'])
    tokens = tokenizer.convert_ids_to_tokens(test_data[i]["input_ids"])
    true_labels = test_data[i]['labels']
    get_labeled_answers(true_labels, tokens)
    get_labeled_answers(out[0], tokens)
    # get_predicted_answers(out[0], tokens)
    # num_answers, num_exact_matches = compare_labels_and_output(out[0], true_labels, tokens)
    
    for idx, pred_label in enumerate(out[0]):
        true_label = true_labels[idx]
        if true_label > 0:
            num_pos_data += 1
        if pred_label > 0:
            # print('label: ', pred_label)
            # print('token: ', tokens[idx])
            num_predicted += 1
            if pred_label == true_label:
                num_correct += 1

# calculate precision and recall
pr = num_correct/num_predicted
rec = num_correct/num_pos_data
print('precision: ', pr)
print('recall: ', rec)


Num test instances:  295
test idx:  0
instance loss:  tensor(1.1996, grad_fn=<NllLossBackward>)
[(20, 13), (92, 22), (149, 25)]
Answers:  ['När du ska hämta ditt beslut ska du ta med ditt pass .', 'Uppehåll ##stillstånd ##skort ##et tillverkas i samband med att beslutet fattas , dock tidigast tre månader innan uppehåll ##stillståndet börjar gälla .', 'Det kan ta upp till fyra veckor att tillverka och leverera kortet till ambassaden eller generalkonsul ##atet efter att du har fått ditt beslut .']
[(1, 2), (3, 2), (5, 15), (20, 13), (33, 17), (50, 17), (67, 1), (68, 24), (92, 1), (93, 1), (94, 20), (114, 12), (126, 23), (149, 25), (174, 20), (194, 22), (216, 27), (243, 17), (260, 1), (263, 29), (292, 4), (314, 1), (351, 2), (370, 3)]
Answers:  ['Eget företag', 'Efter beslut', 'Beslutet skickas till den ambassad eller generalkonsul ##at som du valde i webb ##ansökan .', 'När du ska hämta ditt beslut ska du ta med ditt pass .', 'Du kan få uppehåll ##stillstånd för två år men aldrig längre 